In [ ]:
# Считайте csv файл согласно вашему варианту. 
# Структура файла имеет следующий вид по колонкам в порядке их следования: 
# порядковый номер, имя, фамилия, возраст, доход, номер телефона.

# Необходимо выполнить следующие действия:
# 1)	Удалить колонку с номером телефона
# 2)	Рассчитайте средний доход по данным, а затем отфильтруйте те строки, доход которых меньше среднего.
# 3)	Также примените фильтр по колонке возраст, оставив строки со значением более 25 + (вариант mod10).

# Запишите полученные результаты в новый файл csv, произведя при этом сортировку по полю номер (по возрастанию):
      # заголовок для примера
      # id,name,age,salary
      # 1, Connor John, 25, 1000₽

In [1]:
import csv

In [32]:
filename = "text_4_var_17"

summa = 0
new_list = []

with open('text_4_var_17', newline='\n', encoding='utf-8') as f:
    data = csv.reader(f, delimiter=',')
    
    # убираем колонку с номером телефона
    for i in data:
        dict = {
            "id": int(i[0]),
            "name": i[2] + ' ' + i[1],
            "age": int(i[3]),
            "salary": int(i[4][:-1])
        }
        
        # сумма всего дохода
        summa += dict["salary"]
        new_list.append(dict)
    avage = summa/len(new_list)
    
    # фильтр по доходу ниже среднего и по возрасту 25 + (17mod10)
    filtr_list = []
    filtr_age = 25 + (17 % 10)
    for j in new_list:
        if j['salary'] >= avage and j['age'] > filtr_age:
            filtr_list.append(j)
            
# фильтруем по возрастанию полученные данные
finish_resulte = sorted(filtr_list, key = lambda x:x['id'])
print(finish_resulte)

[{'id': 6, 'name': 'Волкова Матвей', 'age': 50, 'salary': 63726}, {'id': 11, 'name': 'Жуков Маргарита', 'age': 51, 'salary': 87881}, {'id': 13, 'name': 'Петрова Антон', 'age': 76, 'salary': 85334}, {'id': 15, 'name': 'Дьяконов Элина', 'age': 60, 'salary': 80122}, {'id': 17, 'name': 'Антонов Николь', 'age': 63, 'salary': 74570}, {'id': 18, 'name': 'Родионова Элина', 'age': 48, 'salary': 92841}, {'id': 19, 'name': 'Лазарев Оливия', 'age': 41, 'salary': 54938}, {'id': 20, 'name': 'Попова Матвей', 'age': 71, 'salary': 96864}, {'id': 24, 'name': 'Синицын Кирилл', 'age': 63, 'salary': 55887}, {'id': 26, 'name': 'Прохорова Амина', 'age': 72, 'salary': 99726}, {'id': 28, 'name': 'Игнатьев Илья', 'age': 36, 'salary': 56041}, {'id': 29, 'name': 'Демидов Константин', 'age': 51, 'salary': 90152}, {'id': 34, 'name': 'Максимов Антон', 'age': 54, 'salary': 78881}, {'id': 35, 'name': 'Дмитриев Макар', 'age': 47, 'salary': 88370}, {'id': 41, 'name': 'Панова Алексей', 'age': 60, 'salary': 98224}, {'id':

In [28]:
with open(filename+"result", 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f, delimiter=',', quoting=csv.QUOTE_MINIMAL)

    for k in finish_resulte:
        writer.writerow(k.values())